In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd drive/My Drive

Mounted at /content/drive
/content/drive/My Drive


In [95]:
import random
from datetime import datetime

import numpy as np
import pandas as pd
import warnings
import numpy as np
import pandas as pd
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
from sklearn.preprocessing import MinMaxScaler
from tqdm import tqdm


warnings.filterwarnings('ignore')
random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

DEVICE = 'cuda:0' if torch.cuda.is_available() else 'cpu'

train_data = pd.read_csv('data/open data/train.csv').drop([379, 24598], axis=0)
test_data = pd.read_csv('data/open data/test_x.csv')

In [ ]:
train_data.head()

,QaA,QaE,QbA,QbE,QcA,QcE,QdA,QdE,QeA,QeE,QfA,QfE,QgA,QgE,QhA,QhE,QiA,QiE,QjA,QjE,QkA,QkE,QlA,QlE,QmA,QmE,QnA,QnE,QoA,QoE,QpA,QpE,QqA,QqE,QrA,QrE,QsA,QsE,QtA,QtE,age_group,education,engnat,familysize,gender,hand,married,race,religion,tp01,tp02,tp03,tp04,tp05,tp06,tp07,tp08,tp09,tp10,urban,voted,wf_01,wf_02,wf_03,wr_01,wr_02,wr_03,wr_04,wr_05,wr_06,wr_07,wr_08,wr_09,wr_10,wr_11,wr_12,wr_13
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,3.0,363,4.0,1370,5.0,997,1.0,1024,2.0,1577,5.0,539,2.0,586,4.0,1095,5.0,1142,4.0,1287,4.0,883,4.0,851,2.0,851,5.0,816,2.0,579,2.0,924,2.0,366,2.0,876,2.0,633,1.0,1115,30s,2,1,4,Female,1,3,White,Other,2,2,2,1,2,1,7,4,4,3,1,2,0,0,0,0,1,0,0,1,0,1,1,0,1,0,1,1
1,5.0,647,5.0,1313,3.0,3387,5.0,2969,1.0,4320,3.0,2190,1.0,826,1.0,4082,5.0,1867,3.0,1264,5.0,2943,4.0,3927,1.0,4329,5.0,1828,1.0,1214,5.0,2414,5.0,1356,1.0,3039,4.0,4304,1.0,1346,20s,4,2,3,Female,1,1,Asian,Hindu,1,1,0,0,1,2,3,4,0,4,3,2,0,0,0,0,1,0,1,1,0,1,1,0,1,0,1,1
2,4.0,1623,1.0,1480,1.0,1021,4.0,3374,5.0,1333,1.0,531,4.0,1167,1.0,1016,3.0,2653,2.0,1569,5.0,998,5.0,2547,2.0,918,4.0,2153,2.0,1304,1.0,1131,5.0,937,4.0,1327,1.0,1170,1.0,1409,30s,3,1,3,Male,1,2,White,Other,2,3,1,5,3,4,2,6,1,3,2,1,0,0,1,1,1,0,1,1,0,1,1,1,1,0,1,1
3,3.0,504,3.0,2311,4.0,992,3.0,3245,1.0,357,2.0,1519,4.0,159,3.0,2275,5.0,2809,4.0,5614,3.0,3219,4.0,1296,4.0,9046,4.0,1216,4.0,1169,4.0,23868,3.0,581,4.0,8830,4.0,2392,5.0,1312,20s,4,2,0,Female,1,1,Asian,Hindu,2,4,1,1,1,3,1,3,1,3,3,1,0,0,0,0,1,0,0,0,0,0,1,0,1,0,1,1
4,1.0,927,1.0,707,5.0,556,2.0,1062,1.0,1014,2.0,628,1.0,991,1.0,1259,5.0,1153,5.0,1388,5.0,740,5.0,1181,4.0,547,2.0,575,1.0,754,4.0,1140,5.0,323,5.0,1070,1.0,583,2.0,1889,20s,3,1,2,Male,1,2,White,Agnostic,1,1,1,6,0,2,0,6,2,6,1,1,0,1,0,1,1,0,1,1,1,1,1,0,1,1,1,1


In [96]:
flipping_columns = ["QeA", "QfA", "QkA", "QqA", "QrA"]
for flip in flipping_columns: 
    train_data[flip] = 6 - train_data[flip]
    test_data[flip] = 6 - test_data[flip]

flipping_secret_columns = ["QaA", "QdA", "QgA", "QiA", "QnA"]
for flip in flipping_secret_columns: 
    train_data[flip] = 6 - train_data[flip]
    test_data[flip] = 6 - test_data[flip]

Answers = ['QaA', 'QbA', 'QcA', 'QdA', 'QeA',
             'QfA', 'QgA', 'QhA', 'QiA', 'QjA', 
             'QkA', 'QlA', 'QmA', 'QnA', 'QoA', 
             'QpA', 'QqA', 'QrA', 'QsA', 'QtA']

train_data['Mach_score'] = train_data[Answers].mean(axis = 1)
test_data['Mach_score'] = test_data[Answers].mean(axis = 1)


drop_list = ['QaE', 'QbE', 'QcE', 'QdE', 'QeE',
             'QfE', 'QgE', 'QhE', 'QiE', 'QjE',
             'QkE', 'QlE', 'QmE', 'QnE', 'QoE',
             'QpE', 'QqE', 'QrE', 'QsE', 'QtE',
             'index', 'hand']

minmax = MinMaxScaler()

log_Answer_time = train_data[drop_list[:-2]].copy()
log_Answer_time[drop_list[:-2]] = np.log1p(train_data[drop_list[:-2]])

log_Answer_time_t = test_data[drop_list[:-2]].copy()
log_Answer_time_t[drop_list[:-2]] = np.log1p(test_data[drop_list[:-2]])


replace_dict = {'education': str, 'engnat': str, 'married': str, 'urban': str}
train_y = train_data['voted']
train_x = train_data.drop(drop_list + ['voted'], axis=1)
test_x = test_data.drop(drop_list, axis=1)
minmax = MinMaxScaler()
train_x[drop_list[:-2]] = minmax.fit_transform(log_Answer_time)
minmax = MinMaxScaler()
test_x[drop_list[:-2]] = minmax.fit_transform(log_Answer_time_t)
train_x = train_x.astype(replace_dict)
test_x = test_x.astype(replace_dict)
train_x = pd.get_dummies(train_x)
test_x = pd.get_dummies(test_x)
train_y = 2 - train_y.to_numpy()
all_columns = train_x.columns
train_x = train_x.to_numpy()
test_x = test_x.to_numpy()

train_x[:, :20] = (train_x[:, :20] - 3.) / 2.
test_x[:, :20] = (test_x[:, :20] - 3.) / 2
train_x[:, 47] = (train_x[:, 47] - 3.) / 2.
test_x[:, 47] = (test_x[:, 47] - 3.) / 2
train_x[:, 20] = (train_x[:, 20] - 5.) / 5.
test_x[:, 20] = (test_x[:, 20] - 5.) / 5.
train_x[:, 21:31] = (train_x[:, 21:31] - 3.5) / 3.5
test_x[:, 21:31] = (test_x[:, 21:31] - 3.5) / 3.5
train_y = torch.tensor(train_y, dtype=torch.float32)
train_x = torch.tensor(train_x, dtype=torch.float32)
test_x = torch.tensor(test_x, dtype=torch.float32)
train_len, test_len = len(train_x), len(test_x)

In [97]:
all_columns

Index(['QaA', 'QbA', 'QcA', 'QdA', 'QeA', 'QfA', 'QgA', 'QhA', 'QiA', 'QjA',
       ...
       'religion_Christian_Protestant', 'religion_Hindu', 'religion_Jewish',
       'religion_Muslim', 'religion_Other', 'religion_Sikh', 'urban_0',
       'urban_1', 'urban_2', 'urban_3'],
      dtype='object', length=112)

In [98]:
train_len

45530

In [99]:
N_MODEL = 10
N_EPOCH = 50
BATCH_SIZE = 256
LOADER_PARAM = {
    'batch_size': BATCH_SIZE,
    'num_workers': 4,
    'pin_memory': True
}
prediction = np.zeros((11383, 1), dtype=np.float32)
predict = np.zeros((11383, 10), dtype=np.float32)
nn.LeakyReLU
models = []

for no in range(N_MODEL):

    train_loader = DataLoader(TensorDataset(train_x, train_y),
                              shuffle=True, drop_last=True, **LOADER_PARAM)
    test_loader = DataLoader(TensorDataset(test_x, torch.zeros((test_len,), dtype=torch.float32)),
                             shuffle=False, drop_last=False, **LOADER_PARAM)
    model = nn.Sequential(
        nn.Dropout(0.05),
        nn.Linear(112, 160, bias=False),
        nn.BatchNorm1d(160),
        nn.LeakyReLU(0.05, inplace=True),
        nn.Dropout(0.3),
        nn.Linear(160, 224, bias=False),
        nn.BatchNorm1d(224),
        nn.LeakyReLU(0.05, inplace=True),
        nn.Dropout(0.3),
        nn.Linear(224, 184, bias=False),
        nn.BatchNorm1d(184),
        nn.LeakyReLU(0.05, inplace=True),
        nn.Dropout(0.2),
        nn.Linear(184,144 , bias=False),
        nn.BatchNorm1d(144),
        nn.LeakyReLU(0.05, inplace=True),
        nn.Dropout(0.2),
        nn.Linear(144,96 , bias=False),
        nn.BatchNorm1d(96),
        nn.LeakyReLU(0.05, inplace=True),
        nn.Dropout(0.2),
        nn.Linear(96, 36, bias=False),
        nn.LeakyReLU(0.05, inplace=True),
        nn.Linear(36, 1)
    ).to(DEVICE)
    criterion = torch.nn.BCEWithLogitsLoss(pos_weight=torch.tensor([1.20665], device=DEVICE))
    optimizer = optim.AdamW(model.parameters(), lr=1e-3, weight_decay=4e-2)
    scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(
        optimizer, T_0=N_EPOCH // 4, eta_min=1.2e-5)

    model.train()
    for epoch in tqdm(range(N_EPOCH), desc='{:02d}/{:02d}'.format(no + 1, N_MODEL)):
        for idx, (xx, yy) in enumerate(train_loader):
            optimizer.zero_grad()
            xx, yy = xx.to(DEVICE), yy.to(DEVICE)
            pred = model(xx).squeeze()
            loss = criterion(pred, yy)
            loss.backward()
            optimizer.step()
            scheduler.step(epoch + idx / len(train_loader))

    model.eval()
    with torch.no_grad():
        for idx, (xx, _) in enumerate(test_loader):
            xx = xx.to(DEVICE)
            pred = (2. - torch.sigmoid(model(xx).detach().to('cpu'))).numpy()
            prediction[BATCH_SIZE * idx:min(BATCH_SIZE * (idx + 1), len(prediction)), :] += pred[:, :] / N_MODEL
            predict[BATCH_SIZE * idx:min(BATCH_SIZE * (idx + 1), len(prediction)),no] = pred[:,0]
    models.append(model)
# df = pd.read_csv('data/open data/sample_submission.csv')
# df.iloc[:, 1:] = prediction
# df.to_csv('data/open data/{}.csv'.format(datetime.now().strftime('%m%d-%H%M')), index=False)



01/10:   0%|          | 0/50 [00:00<?, ?it/s]

01/10:   2%|▏         | 1/50 [00:01<01:33,  1.90s/it]

01/10:   4%|▍         | 2/50 [00:03<01:31,  1.91s/it]

01/10:   6%|▌         | 3/50 [00:05<01:31,  1.94s/it]

01/10:   8%|▊         | 4/50 [00:07<01:28,  1.92s/it]

01/10:  10%|█         | 5/50 [00:09<01:26,  1.93s/it]

01/10:  12%|█▏        | 6/50 [00:11<01:20,  1.84s/it]

01/10:  14%|█▍        | 7/50 [00:12<01:15,  1.76s/it]

01/10:  16%|█▌        | 8/50 [00:14<01:11,  1.70s/it]

01/10:  18%|█▊        | 9/50 [00:16<01:09,  1.70s/it]

01/10:  20%|██        | 10/50 [00:17<01:06,  1.67s/it]

01/10:  22%|██▏       | 11/50 [00:19<01:04,  1.64s/it]

01/10:  24%|██▍       | 12/50 [00:21<01:02,  1.66s/it]

01/10:  26%|██▌       | 13/50 [00:22<01:01,  1.66s/it]

01/10:  28%|██▊       | 14/50 [00:24<00:58,  1.63s/it]

01/10:  30%|███       | 15/50 [00:26<00:59,  1.70s/it]

01/10:  32%|███▏      | 16/50 [00:27<00:57,  1.70s/it]

01/10:  34%|███▍      | 17/50 [00:29<00:57,  1.75s/it]

01/10:  

In [100]:
for i, model in enumerate(models):
  torch.save(model,'data/open data/model/model{}.pt'.format(i))

In [101]:
import os
models = []
for i in os.listdir('data/open data/model'):
  models.append(torch.load('data/open data/model/' + i))

In [102]:
df = pd.read_csv('data/open data/sample_submission.csv')
df.iloc[:, 1:] = prediction
df.to_csv('data/open data/{}.csv'.format(datetime.now().strftime('%m%d-%H%M')), index=False)

In [62]:
prediction = np.zeros((45440, 1), dtype=np.float32)


for i, model in enumerate(models):
  train_loader = DataLoader(TensorDataset(train_x, train_y),
                              shuffle=True, drop_last=True, **LOADER_PARAM)

  with torch.no_grad():
      for idx, (xx, _) in enumerate(train_loader):
          xx = xx.to(DEVICE)
          pred = (2. - torch.sigmoid(model(xx).detach().to('cpu'))).numpy()
          prediction[BATCH_SIZE * idx:min(BATCH_SIZE * (idx + 1), len(prediction)), :] += pred[:, :] / N_MODEL

In [104]:
len(np.where((prediction < 1.2) | (prediction > 1.8))[0])

3060

In [35]:
pres = np.round(prediction)
pres = np.mean(pres,axis=1)

In [106]:
train_x1 = torch.cat((train_x, test_x[np.where((prediction < 1.2) | (prediction > 1.8))[0],:]), dim=0)
test_pre = 2 - pres[np.where((prediction < 1.2) | (prediction > 1.8))[0]]
test_pre = torch.tensor(test_pre, dtype=torch.float32)
train_y1 = torch.cat((train_y, test_pre))

In [107]:
train_predict = np.zeros((45312, 1), dtype=np.float32)
real = np.zeros((45312, 1), dtype=np.float32)
train_loader = DataLoader(TensorDataset(train_x, train_y),
                              shuffle=True, drop_last=True, **LOADER_PARAM)
test_loader = DataLoader(TensorDataset(test_x, torch.zeros((test_len,), dtype=torch.float32)),
                             shuffle=False, drop_last=False, **LOADER_PARAM)

with torch.no_grad():
    for idx, (xx, _) in enumerate(train_loader):
        xx = xx.to(DEVICE)
        pred = (2. - torch.sigmoid(model(xx).detach().to('cpu'))).numpy()
        train_predict[BATCH_SIZE * idx:min(BATCH_SIZE * (idx + 1), len(train_x)), :] = pred[:, :]
        real[BATCH_SIZE * idx:min(BATCH_SIZE * (idx + 1), len(train_x)),0] = 2 - _[:]
    print(BATCH_SIZE * (idx + 1))

45312


In [108]:
pd.crosstab(pd.Series(real.flat), pd.Series(np.round(train_predict).flat), rownames=['True'], colnames=['Predicted'], margins=True)

Predicted,1.0,2.0,All
True,,,
1.0,17999,2532,20531
2.0,9789,14992,24781
All,27788,17524,45312


In [74]:
train_x2 = train_x[np.where((prediction > 1.3) & (prediction < 1.7))[0],:]
train_y2 = train_y[np.where((prediction > 1.3) & (prediction < 1.7))[0]]

In [75]:
len(train_y2)

44714

In [109]:
N_MODEL = 10
N_EPOCH = 10
BATCH_SIZE = 256
models1 = []
LOADER_PARAM = {
    'batch_size': BATCH_SIZE,
    'num_workers': 4,
    'pin_memory': True
}
prediction = np.zeros((11383, 1), dtype=np.float32)
# predict = np.zeros((11383, 10), dtype=np.float32)

for no, model in enumerate(models):

    train_loader = DataLoader(TensorDataset(train_x1, train_y1),
                              shuffle=True, drop_last=True, **LOADER_PARAM)
    test_loader = DataLoader(TensorDataset(test_x, torch.zeros((test_len,), dtype=torch.float32)),
                             shuffle=False, drop_last=False, **LOADER_PARAM)
    
    criterion = torch.nn.BCEWithLogitsLoss(pos_weight=torch.tensor([1.20665], device=DEVICE))
    optimizer = optim.AdamW(model.parameters(), lr=1e-3, weight_decay=4e-2)
    scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(
        optimizer, T_0=N_EPOCH // 4, eta_min=1.2e-5)

    model.train()
    for epoch in tqdm(range(N_EPOCH), desc='{:02d}/{:02d}'.format(no + 1, N_MODEL)):
        for idx, (xx, yy) in enumerate(train_loader):
            optimizer.zero_grad()
            xx, yy = xx.to(DEVICE), yy.to(DEVICE)
            pred = model(xx).squeeze()
            loss = criterion(pred, yy)
            loss.backward()
            optimizer.step()
            scheduler.step(epoch + idx / len(train_loader))

    model.eval()
    with torch.no_grad():
        for idx, (xx, _) in enumerate(test_loader):
            xx = xx.to(DEVICE)
            pred = (2. - torch.sigmoid(model(xx).detach().to('cpu'))).numpy()
            prediction[BATCH_SIZE * idx:min(BATCH_SIZE * (idx + 1), len(prediction)), :] += pred[:, :] / N_MODEL



01/10:   0%|          | 0/10 [00:00<?, ?it/s]

01/10:  10%|█         | 1/10 [00:01<00:16,  1.80s/it]

01/10:  20%|██        | 2/10 [00:03<00:14,  1.76s/it]

01/10:  30%|███       | 3/10 [00:05<00:12,  1.75s/it]

01/10:  40%|████      | 4/10 [00:06<00:10,  1.74s/it]

01/10:  50%|█████     | 5/10 [00:08<00:08,  1.75s/it]

01/10:  60%|██████    | 6/10 [00:10<00:06,  1.73s/it]

01/10:  70%|███████   | 7/10 [00:12<00:05,  1.71s/it]

01/10:  80%|████████  | 8/10 [00:13<00:03,  1.71s/it]

01/10:  90%|█████████ | 9/10 [00:15<00:01,  1.70s/it]

01/10: 100%|██████████| 10/10 [00:17<00:00,  1.73s/it]


02/10:   0%|          | 0/10 [00:00<?, ?it/s]

02/10:  10%|█         | 1/10 [00:01<00:15,  1.67s/it]

02/10:  20%|██        | 2/10 [00:03<00:13,  1.68s/it]

02/10:  30%|███       | 3/10 [00:05<00:12,  1.75s/it]

02/10:  40%|████      | 4/10 [00:06<00:10,  1.74s/it]

02/10:  50%|█████     | 5/10 [00:08<00:08,  1.75s/it]

02/10:  60%|██████    | 6/10 [00:10<00:07,  1.77s/it]

02/10:  70%|███████   

In [77]:
pres = np.round(prediction)
pres = np.mean(pres,axis=1)
len(np.where((prediction > 1.3) & (prediction < 1.7))[0])

5386

In [ ]:
train_x1 = torch.cat((train_x, test_x[np.where((prediction < 1.3) | (prediction > 1.7))[0],:]), dim=0)
test_pre = 2 - pres[np.where((prediction < 1.3) | (prediction > 1.7))[0]]
test_pre = torch.tensor(test_pre, dtype=torch.float32)
train_y1 = torch.cat((train_y, test_pre))

In [37]:
N_MODEL = 10
N_EPOCH = 15
BATCH_SIZE = 128
models1 = []
LOADER_PARAM = {
    'batch_size': BATCH_SIZE,
    'num_workers': 4,
    'pin_memory': True
}
prediction = np.zeros((11383, 1), dtype=np.float32)
# predict = np.zeros((11383, 10), dtype=np.float32)

for no, model in enumerate(models):

    train_loader = DataLoader(TensorDataset(train_x1, train_y1),
                              shuffle=True, drop_last=True, **LOADER_PARAM)
    test_loader = DataLoader(TensorDataset(test_x, torch.zeros((test_len,), dtype=torch.float32)),
                             shuffle=False, drop_last=False, **LOADER_PARAM)
    
    criterion = torch.nn.BCEWithLogitsLoss(pos_weight=torch.tensor([1.20665], device=DEVICE))
    optimizer = optim.AdamW(model.parameters(), lr=1e-3, weight_decay=4e-2)
    scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(
        optimizer, T_0=N_EPOCH // 4, eta_min=1.2e-5)

    model.train()
    for epoch in tqdm(range(N_EPOCH), desc='{:02d}/{:02d}'.format(no + 1, N_MODEL)):
        for idx, (xx, yy) in enumerate(train_loader):
            optimizer.zero_grad()
            xx, yy = xx.to(DEVICE), yy.to(DEVICE)
            pred = model(xx).squeeze()
            loss = criterion(pred, yy)
            loss.backward()
            optimizer.step()
            scheduler.step(epoch + idx / len(train_loader))

    model.eval()
    with torch.no_grad():
        for idx, (xx, _) in enumerate(test_loader):
            xx = xx.to(DEVICE)
            pred = (2. - torch.sigmoid(model(xx).detach().to('cpu'))).numpy()
            prediction[BATCH_SIZE * idx:min(BATCH_SIZE * (idx + 1), len(prediction)), :] += pred[:, :] / N_MODEL
            # predict[BATCH_SIZE * idx:min(BATCH_SIZE * (idx + 1), len(prediction)),no] = pred[:,0]
df = pd.read_csv('data/open data/sample_submission.csv')
df.iloc[:, 1:] = prediction
df.to_csv('data/open data/{}1.csv'.format(datetime.now().strftime('%m%d-%H%M')), index=False)


01/10: 100%|██████████| 15/15 [00:46<00:00,  3.11s/it]

02/10: 100%|██████████| 15/15 [00:46<00:00,  3.10s/it]

03/10: 100%|██████████| 15/15 [00:45<00:00,  3.06s/it]

04/10: 100%|██████████| 15/15 [00:45<00:00,  3.05s/it]

05/10: 100%|██████████| 15/15 [00:45<00:00,  3.03s/it]

06/10: 100%|██████████| 15/15 [00:45<00:00,  3.04s/it]

07/10: 100%|██████████| 15/15 [00:46<00:00,  3.10s/it]

08/10: 100%|██████████| 15/15 [00:46<00:00,  3.12s/it]

09/10: 100%|██████████| 15/15 [00:47<00:00,  3.14s/it]

10/10: 100%|██████████| 15/15 [00:45<00:00,  3.07s/it]


In [40]:
from sklearn.model_selection import StratifiedKFold


test_len = len(test_x)

N_REPEAT = 5
N_SKFOLD = 5
N_EPOCH = 35
BATCH_SIZE = 256
LOADER_PARAM = {
    'batch_size': BATCH_SIZE,
    'num_workers': 4,
    'pin_memory': True
}
prediction = np.zeros((test_len, 1), dtype=np.float32)

for repeat in range(N_REPEAT):

    skf, tot = StratifiedKFold(n_splits=N_SKFOLD, random_state=repeat, shuffle=True), 0.
    for skfold, (train_idx, valid_idx) in enumerate(skf.split(train_x, train_y)):
        train_idx, valid_idx = list(train_idx), list(valid_idx)
        train_loader = DataLoader(TensorDataset(train_x[train_idx, :], train_y[train_idx]),
                                  shuffle=True, drop_last=True, **LOADER_PARAM)
        valid_loader = DataLoader(TensorDataset(train_x[valid_idx, :], train_y[valid_idx]),
                                  shuffle=False, drop_last=False, **LOADER_PARAM)
        test_loader = DataLoader(TensorDataset(test_x, torch.zeros((test_len,), dtype=torch.float32)),
                                 shuffle=False, drop_last=False, **LOADER_PARAM)
        model = nn.Sequential(
        nn.Dropout(0.05),
        nn.Linear(92, 104, bias=False),
        nn.BatchNorm1d(104),
        nn.LeakyReLU(0.05, inplace=True),
        nn.Dropout(0.2),
        nn.Linear(104, 80, bias=False),
        nn.BatchNorm1d(80),
        nn.LeakyReLU(0.05, inplace=True),
        nn.Linear(80, 64, bias=False),
        nn.BatchNorm1d(64),
        nn.LeakyReLU(0.05, inplace=True),
        nn.Dropout(0.5),
        nn.Linear(64, 32, bias=False),
        nn.BatchNorm1d(32),
        nn.ReLU(inplace=True),
        nn.Linear(32, 1)
        ).to(DEVICE)
        criterion = torch.nn.BCEWithLogitsLoss(pos_weight=torch.tensor([1.20665], device=DEVICE))
        optimizer = optim.AdamW(model.parameters(), lr=5e-3, weight_decay=7.8e-2)
        scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(
            optimizer, T_0=N_EPOCH // 6, eta_min=4e-4)
        prediction_t, loss_t = np.zeros((test_len, 1), dtype=np.float32), 1.

        # for epoch in range(N_EPOCH):
        for epoch in tqdm(range(N_EPOCH), desc='{:02d}/{:02d}'.format(skfold + 1, N_SKFOLD)):
            model.train()
            for idx, (xx, yy) in enumerate(train_loader):
                optimizer.zero_grad()
                xx, yy = xx.to(DEVICE), yy.to(DEVICE)
                pred = model(xx).squeeze()
                loss = criterion(pred, yy)
                loss.backward()
                optimizer.step()
                scheduler.step(epoch + idx / len(train_loader))

            with torch.no_grad():
                model.eval()
                running_acc, running_loss, running_count = 0, 0., 0
                for xx, yy in valid_loader:
                    xx, yy = xx.to(DEVICE), yy.to(DEVICE)
                    pred = model(xx).squeeze()
                    loss = criterion(pred, yy)
                    running_loss += loss.item() * len(yy)
                    running_count += len(yy)
                    running_acc += ((torch.sigmoid(pred) > 0.5).float() == yy).sum().item()
                # print('R{:02d} S{:02d} E{:02d} | {:6.4f}, {:5.2f}%'
                #       .format(repeat + 1, skfold + 1, epoch + 1, running_loss / running_count,
                #               running_acc / running_count * 100))

                if running_loss / running_count < loss_t:
                    loss_t = running_loss / running_count
                    for idx, (xx, _) in enumerate(test_loader):
                        xx = xx.to(DEVICE)
                        pred = (2. - torch.sigmoid(model(xx).detach().to('cpu'))).numpy()
                        prediction_t[BATCH_SIZE * idx:min(BATCH_SIZE * (idx + 1), len(prediction)), :] \
                            = pred[:, :].copy()
        prediction[:, :] += prediction_t[:, :].copy() / (N_REPEAT * N_SKFOLD)
        tot += loss_t
    print('R{} -> {:6.4f}'.format(repeat + 1, tot / N_SKFOLD))



01/05:   0%|          | 0/35 [00:00<?, ?it/s]

01/05:   3%|▎         | 1/35 [00:02<01:10,  2.06s/it]

01/05:   6%|▌         | 2/35 [00:04<01:08,  2.07s/it]

01/05:   9%|▊         | 3/35 [00:06<01:04,  2.03s/it]

01/05:  11%|█▏        | 4/35 [00:07<00:58,  1.90s/it]

01/05:  14%|█▍        | 5/35 [00:09<00:57,  1.93s/it]

01/05:  17%|█▋        | 6/35 [00:11<00:53,  1.84s/it]

01/05:  20%|██        | 7/35 [00:12<00:50,  1.79s/it]

01/05:  23%|██▎       | 8/35 [00:14<00:49,  1.85s/it]

01/05:  26%|██▌       | 9/35 [00:16<00:48,  1.88s/it]

01/05:  29%|██▊       | 10/35 [00:18<00:44,  1.79s/it]

01/05:  31%|███▏      | 11/35 [00:20<00:41,  1.72s/it]

01/05:  34%|███▍      | 12/35 [00:21<00:38,  1.67s/it]

01/05:  37%|███▋      | 13/35 [00:23<00:36,  1.64s/it]

01/05:  40%|████      | 14/35 [00:24<00:35,  1.67s/it]

01/05:  43%|████▎     | 15/35 [00:26<00:33,  1.65s/it]

01/05:  46%|████▌     | 16/35 [00:28<00:31,  1.64s/it]

01/05:  49%|████▊     | 17/35 [00:29<00:29,  1.65s/it]

01/05:  

R1 -> 0.6076




01/05:   3%|▎         | 1/35 [00:01<01:06,  1.96s/it]

01/05:   6%|▌         | 2/35 [00:04<01:06,  2.01s/it]

01/05:   9%|▊         | 3/35 [00:06<01:06,  2.06s/it]

01/05:  11%|█▏        | 4/35 [00:08<01:05,  2.10s/it]

01/05:  14%|█▍        | 5/35 [00:10<01:03,  2.11s/it]

01/05:  17%|█▋        | 6/35 [00:12<00:58,  2.02s/it]

01/05:  20%|██        | 7/35 [00:14<00:54,  1.96s/it]

01/05:  23%|██▎       | 8/35 [00:16<00:51,  1.91s/it]

01/05:  26%|██▌       | 9/35 [00:17<00:48,  1.88s/it]

01/05:  29%|██▊       | 10/35 [00:19<00:46,  1.87s/it]

01/05:  31%|███▏      | 11/35 [00:21<00:44,  1.84s/it]

01/05:  34%|███▍      | 12/35 [00:23<00:41,  1.82s/it]

01/05:  37%|███▋      | 13/35 [00:25<00:39,  1.81s/it]

01/05:  40%|████      | 14/35 [00:26<00:38,  1.81s/it]

01/05:  43%|████▎     | 15/35 [00:28<00:37,  1.88s/it]

01/05:  46%|████▌     | 16/35 [00:30<00:36,  1.94s/it]

01/05:  49%|████▊     | 17/35 [00:33<00:35,  2.00s/it]

01/05:  51%|█████▏    | 18/35 [00:35<00:34,  2.00s/it]


R2 -> 0.6078




01/05:   3%|▎         | 1/35 [00:01<01:05,  1.93s/it]

01/05:   6%|▌         | 2/35 [00:03<01:04,  1.95s/it]

01/05:   9%|▊         | 3/35 [00:06<01:05,  2.03s/it]

01/05:  11%|█▏        | 4/35 [00:07<00:59,  1.92s/it]

01/05:  14%|█▍        | 5/35 [00:09<00:59,  1.97s/it]

01/05:  17%|█▋        | 6/35 [00:11<00:54,  1.87s/it]

01/05:  20%|██        | 7/35 [00:13<00:50,  1.79s/it]

01/05:  23%|██▎       | 8/35 [00:14<00:46,  1.73s/it]

01/05:  26%|██▌       | 9/35 [00:16<00:46,  1.79s/it]

01/05:  29%|██▊       | 10/35 [00:18<00:43,  1.75s/it]

01/05:  31%|███▏      | 11/35 [00:19<00:40,  1.71s/it]

01/05:  34%|███▍      | 12/35 [00:21<00:38,  1.68s/it]

01/05:  37%|███▋      | 13/35 [00:23<00:36,  1.65s/it]

01/05:  40%|████      | 14/35 [00:24<00:35,  1.67s/it]

01/05:  43%|████▎     | 15/35 [00:26<00:32,  1.64s/it]

01/05:  46%|████▌     | 16/35 [00:27<00:30,  1.62s/it]

01/05:  49%|████▊     | 17/35 [00:29<00:29,  1.62s/it]

01/05:  51%|█████▏    | 18/35 [00:31<00:28,  1.68s/it]


R3 -> 0.6077




01/05:   3%|▎         | 1/35 [00:01<01:07,  1.99s/it]

01/05:   6%|▌         | 2/35 [00:03<01:05,  1.99s/it]

01/05:   9%|▊         | 3/35 [00:06<01:04,  2.03s/it]

01/05:  11%|█▏        | 4/35 [00:08<01:02,  2.02s/it]

01/05:  14%|█▍        | 5/35 [00:10<01:00,  2.01s/it]

01/05:  17%|█▋        | 6/35 [00:11<00:54,  1.89s/it]

01/05:  20%|██        | 7/35 [00:13<00:50,  1.80s/it]

01/05:  23%|██▎       | 8/35 [00:14<00:46,  1.73s/it]

01/05:  26%|██▌       | 9/35 [00:16<00:47,  1.83s/it]

01/05:  29%|██▊       | 10/35 [00:18<00:43,  1.76s/it]

01/05:  31%|███▏      | 11/35 [00:20<00:41,  1.72s/it]

01/05:  34%|███▍      | 12/35 [00:21<00:39,  1.72s/it]

01/05:  37%|███▋      | 13/35 [00:23<00:36,  1.68s/it]

01/05:  40%|████      | 14/35 [00:25<00:34,  1.65s/it]

01/05:  43%|████▎     | 15/35 [00:26<00:32,  1.64s/it]

01/05:  46%|████▌     | 16/35 [00:28<00:30,  1.63s/it]

01/05:  49%|████▊     | 17/35 [00:29<00:28,  1.61s/it]

01/05:  51%|█████▏    | 18/35 [00:31<00:27,  1.62s/it]


R4 -> 0.6084




01/05:   3%|▎         | 1/35 [00:02<01:12,  2.13s/it]

01/05:   6%|▌         | 2/35 [00:04<01:08,  2.07s/it]

01/05:   9%|▊         | 3/35 [00:05<01:03,  1.98s/it]

01/05:  11%|█▏        | 4/35 [00:07<01:00,  1.97s/it]

01/05:  14%|█▍        | 5/35 [00:09<00:58,  1.96s/it]

01/05:  17%|█▋        | 6/35 [00:11<00:53,  1.85s/it]

01/05:  20%|██        | 7/35 [00:12<00:49,  1.78s/it]

01/05:  23%|██▎       | 8/35 [00:14<00:47,  1.77s/it]

01/05:  26%|██▌       | 9/35 [00:16<00:44,  1.72s/it]

01/05:  29%|██▊       | 10/35 [00:18<00:45,  1.82s/it]

01/05:  31%|███▏      | 11/35 [00:19<00:41,  1.75s/it]

01/05:  34%|███▍      | 12/35 [00:21<00:38,  1.69s/it]

01/05:  37%|███▋      | 13/35 [00:23<00:36,  1.66s/it]

01/05:  40%|████      | 14/35 [00:24<00:34,  1.63s/it]

01/05:  43%|████▎     | 15/35 [00:26<00:32,  1.61s/it]

01/05:  46%|████▌     | 16/35 [00:27<00:30,  1.60s/it]

01/05:  49%|████▊     | 17/35 [00:29<00:29,  1.63s/it]

01/05:  51%|█████▏    | 18/35 [00:31<00:27,  1.63s/it]


R5 -> 0.6079


In [110]:
df = pd.read_csv('data/open data/sample_submission.csv')
df.iloc[:, 1:] = prediction
df.to_csv('data/open data/{}1.csv'.format(datetime.now().strftime('%m%d-%H%M')), index=False)

In [ ]:
pres1 = np.round(prediction)
pres1 = np.mean(pres1,axis=1)

In [45]:
len(np.where((prediction < 1.25) | (prediction > 1.75))[0])

3710

In [89]:
data = pd.DataFrame()
for i in os.listdir('data/open data'):
  if i.startswith('1') and i.endswith('.csv'):
    dat = pd.read_csv('data/open data/'+i,index_col='index')
    data = pd.concat([data,dat],axis=1)

data.columns = ['v'+str(i) for i in range(1,7)]

In [93]:
k = data.mean(axis=1)
df = pd.read_csv('data/open data/sample_submission.csv')
df.iloc[:, 1:] = k
df.to_csv('data/open data/{}2.csv'.format(datetime.now().strftime('%m%d-%H%M')), index=False)

In [90]:
for i in os.listdir('data/open data'):
  if i.startswith('1') and i.endswith('.csv'):
    print(i)

1114-1245.csv
1114-14141.csv
1116-05391.csv
1116-06011.csv
1116-07111.csv
1116-07381.csv
